<a href="https://colab.research.google.com/github/SergeyShchus/Satellite-Imagery-Analysis/blob/master/%5Cjupyter-notebooksVisualizing_Sentinel_1_as_RGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap

In [2]:
Map = geemap.Map(center=(40, -100), zoom=4)
Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=gsZ_BzynDcH5V4qcQsTt9BZeEW4VTTXtltyi5yYkxxY&tc=6vXhHbzeIHTCrYgNaXInU_1rngNJR42BDlclVO5MBaY&cc=kkkHQXretWUJWxG2H_RhtxL0MmzHuyayfIcLYL3gO_E

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkzrRLf4ngtquo4amFOJTir5jR0GGeAcfaMAzCIhnajCzci3PLMdlQ

Successfully saved authorization token.


Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [3]:
Map.draw_features
roi = ee.FeatureCollection(Map.draw_features)

In [4]:
img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .filterBounds(roi)
)

img_vh = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VH')
    .filterBounds(roi)
)

In [5]:
print(img_vv.size().getInfo()); print(img_vh.size().getInfo())

1675
1675


In [9]:
VH1 = ee.Image(img_vh.filterDate('2023-01-01', '2023-03-30').median())
VH2 = ee.Image(img_vh.filterDate('2023-04-01', '2023-06-30').median())
VH3 = ee.Image(img_vh.filterDate('2023-07-01', '2023-12-30').median())

VV1 = ee.Image(img_vv.filterDate('2023-01-01', '2023-03-30').median())
VV2 = ee.Image(img_vv.filterDate('2023-04-01', '2023-06-30').median())
VV3 = ee.Image(img_vv.filterDate('2023-07-01', '2023-12-30').median())

In [10]:
VH1.bandNames()

In [14]:
vizParams = {
    'bands':'VH',
    'min':[-27],
    'max':[-16]
}
Map.add_layer(VH1.clip(roi), vizParams, 'RGB_VH');
Map

Map(bottom=92521.0, center=[46.98306306632431, 35.32516479492188], controls=(WidgetControl(options=['position'…

In [17]:
VV1.bandNames()

In [19]:
vizParams = {
    'bands':'VV',
    'min':[-18],
    'max':[-7]
}
Map.add_layer(VV1.clip(roi), vizParams, 'RGB_VV');
Map

Map(bottom=184793.0, center=[46.95893298684206, 35.30902862548829], controls=(WidgetControl(options=['position…

In [27]:
#get median from the each of the season
VH1 = ee.Image(img_vh.filterDate('2023-01-01', '2023-03-30').median())
VH2 = ee.Image(img_vh.filterDate('2023-04-01', '2023-06-30').median())
VH3 = ee.Image(img_vh.filterDate('2023-07-01', '2023-12-30').median())

VV1 = ee.Image(img_vv.filterDate('2023-01-01', '2023-03-30').median())
VV2 = ee.Image(img_vv.filterDate('2023-04-01', '2023-06-30').median())
VV3 = ee.Image(img_vv.filterDate('2023-07-01', '2023-12-30').median())

#create a three band stack by creating a new image and adding all bands to the image
VH_RGB = VH1.addBands(VH2).addBands(VH3)
VV_RGB = VV1.addBands(VV2).addBands(VV3)

#assign visualisation parameters
vizParamsVH = {
    'bands':['VH', 'VH_1', 'VH_2'],
    'min':[-27],
    'max':[-16]
}
vizParamsVV = {
    'bands':['VV', 'VV_1', 'VV_2'],
    'min':[-18],
    'max':[-7]
}
Map.addLayer(VH_RGB.clip(roi), vizParamsVH, 'RGB_VH_Composite');
Map.addLayer(VV_RGB.clip(roi), vizParamsVV, 'RGB_VV_Composite');
Map

Map(bottom=92542.0, center=[46.96338498544958, 35.28396606445313], controls=(WidgetControl(options=['position'…

In [29]:
#Create a ratio image by substracting VV1 from VH1
ratio = (VH1).subtract(VV1).rename('Ratio')

#Generate a new image by adding three bands to the image
rgb_ratio = VH1.addBands(VV1).addBands(ratio)
rgb_ratio.bandNames()

#Specify visualization parameters for each band
vizParams = {
    'bands':['VH', 'VV', 'Ratio'],
    'min':[-26, -17, -12],
    'max':[-23, -10, -6]
}

#Add the map layer and enjoy the colorful map
Map.addLayer(rgb_ratio.clip(roi), vizParams, 'RGB_VVVHRatio')
Map

Map(bottom=1479307.0, center=[46.77943245969339, 35.115609169006355], controls=(WidgetControl(options=['positi…